In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from spe.mse_estimator import ErrorComparer, gen_rbf_X
from spe.forest import BlurredForest

from scipy.linalg import toeplitz

In [2]:
err_cmp = ErrorComparer()

In [21]:
niter = 100
n_estimators = 10

n=200
p=300
s=30

rho = 0.5
c = np.zeros(n)
c[:20] = [rho**i for i in np.arange(20)] 
Sigma_t = toeplitz(c)
Chol_t = np.linalg.cholesky(Sigma_t)
# Chol_t = None

# Theta_p = np.linalg.inv(Sigma_t)
# Theta_p = np.linalg.inv(Chol_t @ Chol_t.T)
# Theta_p = np.diag(np.random.choice([0,1],size=n,replace=True))
Theta_p = None

alphas = [0.05, 0.1, 0.2, 0.5, 0.8, 1.]
# alphas = [0.05, 0.5, 1.]
na = len(alphas)

In [22]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()

# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

X = gen_rbf_X(c_x, c_y, p)
# X = np.random.randn(n,p)
beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [23]:
test_err = np.zeros((na, niter))
blur_err = np.zeros((na, niter))

for i,alpha in enumerate(alphas):
    print(i)
    (test_err[i,:],
     blur_err[i,:]) = err_cmp.compareForest(n=n,
                                         p=p,
                                         X=X,
                                         beta=beta,
                                         Chol_t=Chol_t,
                                         Theta_p=Theta_p,
                                         model=BlurredForest(max_depth=4, 
                                                             max_features="sqrt",
                                                             n_estimators=n_estimators),
                                         alpha=alpha,
                                         niter=niter,
                                         use_expectation=True,
                                         rand_type='full',
                                         est_risk=False)
    

0


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 196 is different from 200)

In [ ]:
risk = test_err.mean(axis=1)

In [24]:
blur_df = pd.DataFrame((blur_err.T))# - test_err.T)/test_err.T)

In [25]:
fig_blur = px.box((blur_df - test_err.mean(1))/test_err.mean(1), 
                  labels={
                         "variable": "Alpha",
                         "value": "Risk"
                         },
                  points='all',
                  title=f"Blurred full refit n={n}, p={p}")
fig_blur.update_traces(boxmean=True)
# fig_blur.add_trace(go.Scatter(x=[-1,na], 
#                          y=[test_err.mean(),test_err.mean()], 
#                          mode='lines', 
#                          name=f'err: {test_err.mean():.2f}'))
fig_blur.add_hline(y=0.,line_color='red')
fig_blur.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(na),
        ticktext = alphas,
    )
)

In [26]:
test_err_rand = np.zeros((na, niter))
blur_err_rand = np.zeros((na, niter))

X = np.random.randn(n,p)
beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

for i,alpha in enumerate(alphas):
    print(i)
    (test_err_rand[i,:],
     blur_err_rand[i,:]) = err_cmp.compareForest(n=n,
                                         p=p,
                                         X=X,
                                         beta=beta,
                                         Chol_t=Chol_t,
                                         Theta_p=Theta_p,
                                         model=BlurredForest(max_depth=4, 
                                                             max_features="sqrt",
                                                             n_estimators=n_estimators),
                                         alpha=alpha,
                                         niter=niter,
                                         use_expectation=False,
                                         rand_type='full',
                                         est_risk=False)
    

0
1
2
3
4
5


In [27]:
risk = test_err_rand.mean(axis=1)

In [28]:
blur_df_rand = pd.DataFrame((blur_err_rand.T))# - test_err_rand.T)/test_err_rand.T)

In [29]:
fig_blur_rand = px.box((blur_df_rand - test_err_rand.mean(1))/test_err_rand.mean(1), 
                  labels={
                         "variable": "Alpha",
                         "value": "Risk"
                         },
                  points=False,
                  title=f"Blurred full refit rand correction n={n}, p={p}")
fig_blur_rand.update_traces(boxmean=True)
# fig_blur.add_trace(go.Scatter(x=[-1,na], 
#                          y=[test_err_rand.mean(),test_err_rand.mean()], 
#                          mode='lines', 
#                          name=f'err: {test_err_rand.mean():.2f}'))
fig_blur_rand.add_hline(y=0.,line_color='red')
fig_blur_rand.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(na),
        ticktext = alphas,
    )
)

In [30]:
test_err_w = np.zeros((na, niter))
blur_err_w = np.zeros((na, niter))

X = np.random.randn(n,p)
beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

for i,alpha in enumerate(alphas):
    print(i)
    (test_err_w[i,:],
     blur_err_w[i,:]) = err_cmp.compareForest(n=n,
                                         p=p,
                                         X=X,
                                         beta=beta,
                                         Chol_t=Chol_t,
                                         Theta_p=Theta_p,
                                         model=BlurredForest(max_depth=4, 
                                                             max_features="sqrt",
                                                             n_estimators=n_estimators),
                                         alpha=alpha,
                                         niter=niter,
                                         use_expectation=True,
                                         rand_type='rand',
                                         est_risk=False)
    

0
1
2
3
4
5


In [31]:
risk_w = test_err_w.mean(axis=1)

In [32]:
blur_df_w = pd.DataFrame((blur_err_w.T))# - test_err_w.T)/test_err_w.T)

In [33]:
fig_blur_w = px.box((blur_df_w - test_err_w.mean(1))/test_err_w.mean(1), 
                  labels={
                         "variable": "Alpha",
                         "value": "Risk"
                         },
                  points=False,
                  title=f"Blurred W refit n={n}, p={p}")
fig_blur_w.update_traces(boxmean=True)
# fig_blur_w.add_trace(go.Scatter(x=[-1,na], 
#                          y=[test_err_w.mean(),test_err_w.mean()], 
#                          mode='lines', 
#                          name=f'err: {test_err_w.mean():.2f}'))
fig_blur_w.add_hline(y=0.,line_color='red')
fig_blur_w.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(na),
        ticktext = alphas,
    )
)

In [34]:
test_err_w_rand = np.zeros((na, niter))
blur_err_w_rand = np.zeros((na, niter))

X = np.random.randn(n,p)
beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

for i,alpha in enumerate(alphas):
    print(i)
    (test_err_w_rand[i,:],
     blur_err_w_rand[i,:]) = err_cmp.compareForest(n=n,
                                         p=p,
                                         X=X,
                                         beta=beta,
                                         Chol_t=Chol_t,
                                         Theta_p=Theta_p,
                                         model=BlurredForest(max_depth=4, 
                                                             max_features="sqrt",
                                                             n_estimators=n_estimators),
                                         alpha=alpha,
                                         niter=niter,
                                         use_expectation=False,
                                         rand_type='rand',
                                         est_risk=False)
    

0
1
2
3
4
5


In [35]:
risk_w_rand = test_err_w_rand.mean(axis=1)

In [36]:
blur_df_w_rand = pd.DataFrame((blur_err_w_rand.T))# - test_err_w_rand.T)/test_err_w_rand.T)

In [37]:
fig_blur_w_rand = px.box((blur_df_w_rand - test_err_w_rand.mean(1))/test_err_w_rand.mean(1), 
                  labels={
                         "variable": "Alpha",
                         "value": "Risk"
                         },
                  points=False,
                  title=f"Blurred W refit rand correction n={n}, p={p}")
fig_blur_w_rand.update_traces(boxmean=True)
# fig_blur_w.add_trace(go.Scatter(x=[-1,na], 
#                          y=[test_err_w_rand.mean(),test_err_w_rand.mean()], 
#                          mode='lines', 
#                          name=f'err: {test_err_w_rand.mean():.2f}'))
fig_blur_w_rand.add_hline(y=0.,line_color='red')
fig_blur_w_rand.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(na),
        ticktext = alphas,
    )
)